In [ ]:
from downhole.survey import Survey, DownholeSurvey

In [ ]:
from downhole.examples.ex_survey import *

In [ ]:
s = Survey()

In [ ]:
import pandas as pd

In [ ]:
survey_df = pd.read_excel('Survey.xlsx', index_col=0)
collar_df = pd.read_excel('Collar.xlsx', index_col=0)

In [ ]:
holes = list(pd.unique(df.index))
for hole in holes:
    ds = DownholeSurvey(list(df.loc[[hole]]['AzimGridN']),
                        list(df.loc[[hole]]['Final_Dip']),
                        list(df.loc[[hole]]['Depth_m']), x, y, z, hole)
    s.add_downhole_survey(ds)



In [ ]:
s.export_shapefile()

In [ ]:
s.downhole_surveys[1].min_curv()

In [ ]:
collar_df.loc[['KDI-12-001']]
holes = list(pd.unique(collar_df.index))
for hole in holes:
    try:
        if len(survey_df.loc[[hole]]) == 1:
            print(f"No Downhole Survey {hole}")
    except KeyError:
        print(f"Missing {hole}")